# Reference

**Library**

* [librosa](https://librosa.org/doc/main/feature.html)


**Tech Blog**

* [제 06절 아날로그 신호](https://m.blog.naver.com/gkenq/220679236344)
* [AI에게 어떻게 음성을 가르칠까?](https://tech.kakaoenterprise.com/66)
* [음성인식 입문하기](https://pyy0715.github.io/Audio/)
* [음악 데이터 - spectral_centroid, sepectral_rolloff](https://0equal2.tistory.com/144)
* [[Librosa] music/audio processing library Librosa 사용법 Tutorial - (3) Audio feature extraction](https://bo-10000.tistory.com/entry/Librosa-musicaudio-processing-library-Librosa-%EC%82%AC%EC%9A%A9%EB%B2%95-Tutorial-3-Audio-feature-extraction)

* [Tonnetz 음악이론을 딥러닝에 접목시키기](https://inspiringpeople.github.io/data%20analysis/tonnetz-dl/)


**You Tube**

* [DMQA, Introduction to Analysis for Sound Data](https://www.youtube.com/watch?v=1Hhj14QhkaE&t=718s)


**데이콘 코드 공유**
* [mel-spectrogram, mfcc 활용한 CNN ensemble + 5fold / public 0.98](https://dacon.io/en/codeshare/5153)

# Setting

## Library

In [1]:
import pandas as pd
import numpy as np

from scipy import stats

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, TruncatedSVD, IncrementalPCA
from sklearn.decomposition import TruncatedSVD

import os
from tqdm.auto import tqdm
import random

In [2]:
import librosa
import librosa.display
import IPython.display as ipd

In [3]:
import warnings
warnings.filterwarnings(action='ignore') 

## Fixed Random Seed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Load Data Set

## Google Drive Mount

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Unzip File

In [6]:
!unzip -qq '/content/drive/MyDrive/머신러닝 엔지니어링/데이콘/기계 고장 진단/data/기계_고장.zip'

## Load Train / Test Set

In [7]:
df_train = pd.read_csv('./train.csv') # 모두 정상 Sample
df_test = pd.read_csv('./test.csv')

In [8]:
print(df_train.shape)
df_train.head()

(1279, 4)


,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE,LABEL
0,TRAIN_0000,./train/TRAIN_0000.wav,2,0
1,TRAIN_0001,./train/TRAIN_0001.wav,0,0
2,TRAIN_0002,./train/TRAIN_0002.wav,0,0
3,TRAIN_0003,./train/TRAIN_0003.wav,2,0
4,TRAIN_0004,./train/TRAIN_0004.wav,2,0


In [9]:
print(df_test.shape)
df_test.head()

(1514, 3)


,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE
0,TEST_0000,./test/TEST_0000.wav,2
1,TEST_0001,./test/TEST_0001.wav,2
2,TEST_0002,./test/TEST_0002.wav,0
3,TEST_0003,./test/TEST_0003.wav,0
4,TEST_0004,./test/TEST_0004.wav,0


# Feature Engineering

## Scaling

In [10]:
def scaled_df(df_train, df_train_fan_type, df_test, df_test_fan_type, scaler):

  df_train_fan_type = df_train_fan_type[['FAN_TYPE']]

  df_train = pd.concat([
                        df_train.reset_index(drop=True),
                        df_train_fan_type.reset_index(drop=True)
                       ],
                       axis=1)
  
  df_test_fan_type = df_test_fan_type[['FAN_TYPE']]

  df_test = pd.concat([
                       df_test.reset_index(drop=True),
                       df_test_fan_type.reset_index(drop=True)
                      ],
                      axis=1)
  
  train_type_0 = df_train.loc[(df_train['FAN_TYPE']==0)]
  train_type_2 = df_train.loc[(df_train['FAN_TYPE']==2)]

  test_type_0 = df_test.loc[(df_test['FAN_TYPE']==0)]
  test_type_2 = df_test.loc[(df_test['FAN_TYPE']==2)]

  train_type_0.drop(columns='FAN_TYPE', inplace=True)
  train_type_2.drop(columns='FAN_TYPE', inplace=True)
  test_type_0.drop(columns='FAN_TYPE', inplace=True)
  test_type_2.drop(columns='FAN_TYPE', inplace=True)

  list_train_0_index = list(train_type_0.index)
  list_train_2_index = list(train_type_2.index)

  list_test_0_index = list(test_type_0.index)
  list_test_2_index = list(test_type_2.index)

  scaled_train_type_0 = scaler.fit_transform(train_type_0)
  scaled_test_type_0 = scaler.transform(test_type_0)

  scaled_train_type_2 = scaler.fit_transform(train_type_2)
  scaled_test_type_2 = scaler.transform(test_type_2)

  train_type_0 = pd.DataFrame(scaled_train_type_0)
  train_type_2 = pd.DataFrame(scaled_train_type_2)

  test_type_0 = pd.DataFrame(scaled_test_type_0)
  test_type_2 = pd.DataFrame(scaled_test_type_2)

  train_type_0.index = list_train_0_index
  train_type_2.index = list_train_2_index

  test_type_0.index = list_test_0_index
  test_type_2.index = list_test_2_index

  df_train = pd.concat([train_type_0, train_type_2], axis=0)
  df_test = pd.concat([test_type_0, test_type_2], axis=0)

  df_train.sort_index(inplace=True)
  df_test.sort_index(inplace=True)

  return df_train, df_test

## Feature Extraction

**Sound**

* 공기나 물 같은 매질의 진동을 통해 전달되는 종파
* 사람의 귀에 들려오는 소리는 공기로 전해오는 파동

**소리의 3요소**

* 세기 (소리의 크기)
* 높낮이 (소리의 높고 낮음)
* 음색 (소리의 색상)

**아날로그 신호**

* 자연계에 포함되어 있는 연속적인 파형
* 주기 신호와 비주기 신호로 분류할 수 있다

**주기 신호**

* 연속적으로 반복된 패턴으로 구성
* 사이클 (Cycle) - 하나의 완성된 패턴
* 정형파와 비정형파로 분류할 수 있다

**비주기 신호**

* 시간에 따라 반복된 패턴이나 사이클 없이 항상 변한다


**정형파**

1. 진폭 (Amplitude)

  - 신호의 크기나 또는 세기를 나타낸다
  - 신호의 높이
  - 단위 : V (volt)

2. 주기 (Period)와 주파수 (Frequency)

  - 주기 : 하나의 사이클을 완성하는데 필요한 시간
  - 주파수 : 주기의 역수, 1초에 완성되는 주기 횟수
  - 높은 주파수 : 짧은 기간내의 변화
  - 낮은 주파수 : 긴 기간에 걸친 변화
  - 단위 : HZ (cycle / second)

3. 위상 (phase)

  - 임의의 시간에서 반송파 사이클의 상대적인 위치
  - 시간 0에서 대한 파형의 상대적인 위치
  - 시간축을 따라 앞뒤로 이동될 수 있는 파형에서 그 이동된 양
  - 첫 사이클의 상태
  - 단위 : degree

4. 대역폭 (Bandwidth)

  - 복합신호의 대역폭은 신호에 포함된 최고 주파수와 최저 주파수 사이의 차이

**Sampling Rate**

* 소리를 컴퓨터에 입력시키기 위해 음파를 숫자로 표현할 필요가 있음

* 샘플링 레이트는 아날로그 신호를 숫자화 시킨 것


**스펙트럼 (Spectrum)**

* 푸리에 변환을 통해 time domain에서 frequency domain으로 바뀐 그래프
* 특정 시간 길이의 음석 조각 (프레임)이 각각의 주파수 성분들을 얼마칸큼 갖고 있는지 의미

**스펙트로그램 (Spectrogram)**

* 파형 (Waveform)과 스펙트럼 (Spectrum) 조합
* 음향 신호를 주파수, 진폭, 시간으로 구분
* X축 : 시간
* Y축 : 주파수
* Z축 : 진폭



### Zero Crossing Rate


* Compute the zero-crossing rate of an audio time series.
* 신호의 부호가 바뀌는 비율

In [356]:
def get_zero_crossing_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 mfcc 추출
        zero = librosa.feature.zero_crossing_rate(y=y)

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in zero:

            e = np.mean(e)

            y_feature.append(e)

        features.append(y_feature)
    
    zero_df = pd.DataFrame(features,
                           columns=['Zero Crossing Rate'])

    print(zero_df.shape)

    return zero_df

In [357]:
zero_train = get_zero_crossing_feature(df_train)
zero_test = get_zero_crossing_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [358]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
zero_train, zero_test= scaled_df(zero_train,
                                 df_train,
                                 zero_test,
                                 df_test,
                                 scaler)

zero_train.columns = ['Zero_Crossing_Rate']
zero_test.columns = ['Zero_Crossing_Rate']

In [359]:
zero_train.head()

,Zero_Crossing_Rate
0,0.056510
1,-0.545779
2,0.806084
3,-0.547025
4,2.378941


In [360]:
def get_zero_crossing_delta_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 mfcc 추출
        zero = librosa.feature.zero_crossing_rate(y=y)
        zero = librosa.feature.delta(
                                   zero,
                                   order=1
                                   )

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in zero:

            e = np.mean(e)

            y_feature.append(e)

        features.append(y_feature)
    
    zero_df = pd.DataFrame(features,
                           columns=['Zero Crossing Rate delta'])

    print(zero_df.shape)

    return zero_df

In [361]:
zero_delta_train = get_zero_crossing_delta_feature(df_train)
zero_delta_test = get_zero_crossing_delta_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [362]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
zero_delta_train, zero_delta_test= scaled_df(zero_delta_train,
                                 df_train,
                                 zero_delta_test,
                                 df_test,
                                 scaler)

zero_delta_train.columns = ['Zero Crossing Rate delta']
zero_delta_test.columns = ['Zero Crossing Rate delta']

### RMS

* Compute root-mean-square (RMS) value for each frame, either from the audio samples y or from a spectrogram S.

In [363]:
def get_rms_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 RMS 추출
        rms = librosa.feature.rms(y=y)

        y_feature = []
        # 추출된 RMS의 절사평균을 Feature로 사용
        for e in rms:

            e = np.mean(e)

            y_feature.append(e)

        features.append(y_feature)
    
    rms_df = pd.DataFrame(features,
                           columns=['RMS'])

    print(rms_df.shape)

    return rms_df

In [364]:
rms_train = get_rms_feature(df_train)
rms_test = get_rms_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [365]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
rms_train, rms_test= scaled_df(rms_train,
                                 df_train,
                                 rms_test,
                                 df_test,
                                 scaler)

rms_train.columns = ['RMS']
rms_test.columns = ['RMS']

In [366]:
rms_train.head()

,RMS
0,0.914106
1,-0.092390
2,-1.560508
3,1.459444
4,-1.562469


In [367]:
def get_rms_delta_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 RMS 추출
        rms = librosa.feature.rms(y=y)
        rms = librosa.feature.delta(
                                   rms,
                                   order=1
                                   )

        y_feature = []
        # 추출된 RMS의 절사평균을 Feature로 사용
        for e in rms:

            e = np.mean(e)

            y_feature.append(e)

        features.append(y_feature)
    
    rms_df = pd.DataFrame(features,
                           columns=['RMS_delta'])

    print(rms_df.shape)

    return rms_df

In [368]:
rms_delta_train = get_rms_delta_feature(df_train)
rms_delta_test = get_rms_delta_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [369]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
rms_delta_train, rms_delta_test= scaled_df(rms_delta_train,
                                 df_train,
                                 rms_delta_test,
                                 df_test,
                                 scaler)

rms_delta_train.columns = ['RMS_delta']
rms_delta_test.columns = ['RMS_delta']

In [370]:
rms_delta_train.head()

,RMS_delta
0,-1.279658
1,-0.283601
2,-0.532520
3,-0.671733
4,-1.726304


### Poly Feature

* Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.

In [19]:
def get_poly_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        poly = librosa.feature.poly_features(
                                             y=y,
                                             sr=sr,
                                             #order=2
                                             )

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in poly:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Poly'+str(i) for i in range(len(features[0]))]
    
    poly_df = pd.DataFrame(features,
                           columns=columns)

    print(poly_df.shape)

    return poly_df

In [20]:
poly_train = get_poly_feature(df_train)
poly_test = get_poly_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 2)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 2)


In [21]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
poly_train, poly_test= scaled_df(poly_train,
                                 df_train,
                                 poly_test,
                                 df_test,
                                 scaler)

poly_train.columns = ['Poly_'+str(i) for i in range(len(poly_train.columns))]
poly_test.columns = ['Poly_'+str(i) for i in range(len(poly_test.columns))]

In [22]:
poly_train.head()

,Poly_0,Poly_1
0,-1.251050,1.773784
1,0.500273,-1.051736
2,1.853597,-1.256996
3,-1.329506,1.690499
4,1.831355,-1.390130


### Spectral

#### Spectral Centroid

* Compute the spectral centroid.

* Each frame of a magnitude spectrogram is normalized and treated as a distribution over frequency bins, from which the mean (centroid) is extracted per frame.

* 음성의 각 프레임마다 평균(중심) 주파수를 반환

* Spectrum의 질량중심(Center of mass)을 구한다. Frequency들의 magnitude에 따른 centroid들의 weighted sum이라고 생각하면 된다.

In [23]:
def get_spectral_centroid_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in centroid:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_centroid_'+str(i) for i in range(len(features[0]))]
    
    centroid_df = pd.DataFrame(features,
                           columns=columns)

    print(centroid_df.shape)

    return centroid_df

In [24]:
spectral_centroid_train = get_spectral_centroid_feature(df_train)
spectral_centroid_test = get_spectral_centroid_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [25]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
spectral_centroid_train, spectral_centroid_test= scaled_df(spectral_centroid_train,
                                                           df_train,
                                                           spectral_centroid_test,
                                                           df_test,
                                                           scaler)

spectral_centroid_train.columns = ['Spectral_Centroid_'+str(i) for i in range(len(spectral_centroid_train.columns))]
spectral_centroid_test.columns = ['Spectral_Centroid_'+str(i) for i in range(len(spectral_centroid_test.columns))]

In [26]:
spectral_centroid_train.head()

,Spectral_Centroid_0
0,-0.168663
1,-1.391978
2,1.280020
3,-0.577659
4,2.607732


#### Spectral bandwidth

* Compute p’th-order spectral bandwidth.

* Spectrum의 bandwidth를 측정한다. Frequency들의 magnitude에 따른 centroid의 s.t.d.들의 weighted sum이라고 생각하면 된다.

In [27]:
def get_spectral_bandwidth_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in bandwidth:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_bandwidth_'+str(i) for i in range(len(features[0]))]
    
    bandwidth_df = pd.DataFrame(features,
                           columns=columns)

    print(bandwidth_df.shape)

    return bandwidth_df

In [28]:
spectral_bandwidth_train = get_spectral_bandwidth_feature(df_train)
spectral_bandwidth_test = get_spectral_bandwidth_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 1)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 1)


In [29]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
spectral_bandwidth_train, spectral_bandwidth_test= scaled_df(spectral_bandwidth_train,
                                                           df_train,
                                                           spectral_bandwidth_test,
                                                           df_test,
                                                           scaler)

spectral_bandwidth_train.columns = ['Spectral_Bandwidth_'+str(i) for i in range(len(spectral_bandwidth_train.columns))]
spectral_bandwidth_test.columns = ['Spectral_Bandwidth_'+str(i) for i in range(len(spectral_bandwidth_test.columns))]

In [30]:
spectral_bandwidth_train.head()

,Spectral_Bandwidth_0
0,0.715713
1,-0.822927
2,1.332255
3,0.180795
4,0.024884


#### Spectral contrast

* Compute spectral contrast

* Each frame of a spectrogram S is divided into sub-bands. For each sub-band, the energy contrast is estimated by comparing the mean energy in the top quantile (peak energy) to that of the bottom quantile (valley energy). High contrast values generally correspond to clear, narrow-band signals, while low contrast values correspond to broad-band noise.

In [31]:
def get_spectral_contrast_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in contrast:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_contrast_'+str(i) for i in range(len(features[0]))]
    
    contrast_df = pd.DataFrame(features,
                           columns=columns)

    print(contrast_df.shape)

    return contrast_df

In [ ]:
spectral_contrast_train = get_spectral_contrast_feature(df_train)
spectral_contrast_test = get_spectral_contrast_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

In [ ]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
spectral_contrast_train, spectral_contrast_test= scaled_df(spectral_contrast_train,
                                                           df_train,
                                                           spectral_contrast_test,
                                                           df_test,
                                                           scaler)

spectral_contrast_train.columns = ['Spectral_Contrast_'+str(i) for i in range(len(spectral_contrast_train.columns))]
spectral_contrast_test.columns = ['Spectral_Contrast_'+str(i) for i in range(len(spectral_contrast_test.columns))]

In [ ]:
spectral_contrast_train.head()

#### Spectral Flatness

* Compute spectral flatness

* Spectral flatness (or tonality coefficient) is a measure to quantify how much noise-like a sound is, as opposed to being tone-like 1. A high spectral flatness (closer to 1.0) indicates the spectrum is similar to white noise. It is often converted to decibel.

* Spectrum의 noisiness (또는 tonality)를 구한다. Frequency들의 기하평균'(geometric mean)에 대한 산술평균(arithmetic mean)의 비를 이용해 구할 수 있다. 1에 가까울수록 white noise (maximum flatness)를 의미한다.

In [ ]:
def get_spectral_flatness_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        flatness = librosa.feature.spectral_flatness(y=y)

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in flatness:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_flatness_'+str(i) for i in range(len(features[0]))]
    
    flatness_df = pd.DataFrame(features,
                           columns=columns)

    print(flatness_df.shape)

    return flatness_df

In [ ]:
spectral_flatness_train = get_spectral_flatness_feature(df_train)
spectral_flatness_test = get_spectral_flatness_feature(df_test)

In [ ]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
spectral_flatness_train, spectral_flatness_test= scaled_df(spectral_flatness_train,
                                                           df_train,
                                                           spectral_flatness_test,
                                                           df_test,
                                                           scaler)

spectral_flatness_train.columns = ['Spectral_Flatness_'+str(i) for i in range(len(spectral_flatness_train.columns))]
spectral_flatness_test.columns = ['Spectral_Flatness_'+str(i) for i in range(len(spectral_flatness_test.columns))]

In [ ]:
spectral_flatness_train.head()

#### Spectral Rolloff

* Compute roll-off frequency.

* The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and the bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent to a value close to 1 (or 0).

* 스펙트로그렘에서 roll percent 위치에 차지하는 주파수를 구함

In [ ]:
def get_spectral_rolloff_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 poly 추출
        rolloff = librosa.feature.spectral_rolloff(y=y,
                                                   roll_percent=0.15
                                                   )

        y_feature = []
        # 추출된 Poly들의 평균을 Feature로 사용
        for e in rolloff:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['spectral_rolloff_'+str(i) for i in range(len(features[0]))]
    
    rolloff_df = pd.DataFrame(features,
                           columns=columns)

    print(rolloff_df.shape)

    return rolloff_df

In [ ]:
spectral_rolloff_train = get_spectral_rolloff_feature(df_train)
spectral_rolloff_test = get_spectral_rolloff_feature(df_test)

In [ ]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
spectral_rolloff_train, spectral_rolloff_test= scaled_df(spectral_rolloff_train,
                                                           df_train,
                                                           spectral_rolloff_test,
                                                           df_test,
                                                           scaler)

spectral_rolloff_train.columns = ['Spectral_Rolloff_'+str(i) for i in range(len(spectral_rolloff_train.columns))]
spectral_rolloff_test.columns = ['Spectral_Rolloff_'+str(i) for i in range(len(spectral_rolloff_test.columns))]

In [ ]:
spectral_rolloff_train.head()

### Mel Spectrogram

* 사람은 낮은 주파수를 높은 주파수보다 더 예민하게 받아들임

* 500 ~ 1500 Hz 가 바뀌는건 예민하게 인식하는데 반해 10000Hz ~ 11000Hz가 바뀌는 것은 잘 인식 못한다는 것.

* 주파수 단위를 다음 공식에 따라 멜 스케일로 변환 m = 2595 \log_{10}\left(1 + \frac{f}{700}\right)m=2595log10​(1+700f​)

* 해당 scale을 적용해 spectogram을 만든 것이 melspectogram

In [ ]:
def get_melspectrogram_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 melspectrogram 추출
        melspectrogram = librosa.feature.melspectrogram(y=y,
                                                        sr=sr,
                                                        n_fft=2048,
                                                        hop_length=512,
                                                        )

        y_feature = []
        # 추출된 melspectrogram들의 평균을 Feature로 사용
        for e in melspectrogram:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Mel_Spectrogram_'+str(i) for i in range(len(features[0]))]
    
    melspectrogram_df = pd.DataFrame(features,
                           columns=columns)

    print(melspectrogram_df.shape)

    return melspectrogram_df

In [ ]:
melspectrogram_train = get_melspectrogram_feature(df_train)
melspectrogram_test = get_melspectrogram_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 128)


  0%|          | 0/1514 [00:00<?, ?it/s]

In [ ]:
scaler = RobustScaler()
melspectrogram_train, melspectrogram_test= scaled_df(melspectrogram_train,
                                       df_train,
                                       melspectrogram_test,
                                       df_test,
                                       scaler)

melspectrogram_train.columns = ['Mel_Spectrogram_'+str(i) for i in range(len(melspectrogram_train.columns))]
melspectrogram_test.columns = ['Mel_Spectrogram_'+str(i) for i in range(len(melspectrogram_test.columns))]

In [ ]:
melspectrogram_train.head()

### MFCC

* Mel-Spectrogram이라는 피쳐에 대해 행렬을 압축해서 표현해주는 DCT 연산을 수행

In [371]:
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y,
                                    sr=sr,
                                    n_mfcc=128,
                                    dct_type=2)

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:

            e = np.mean(e)

            y_feature.append(e)

        features.append(y_feature)

    columns = ['MFCC_'+str(i) for i in range(len(features[0]))]
    
    mfcc_df = pd.DataFrame(features,
                           columns=columns)

    print(mfcc_df.shape)

    return mfcc_df

In [372]:
mfcc_train = get_mfcc_feature(df_train)
mfcc_test = get_mfcc_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 128)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 128)


In [373]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
mfcc_train, mfcc_test= scaled_df(mfcc_train,
                                 df_train,
                                 mfcc_test,
                                 df_test,
                                 scaler)

mfcc_train.columns = ['MFCC_'+str(i) for i in range(len(mfcc_train.columns))]
mfcc_test.columns = ['MFCC_'+str(i) for i in range(len(mfcc_test.columns))]

In [374]:
mfcc_train.head()

,MFCC_0,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,...,MFCC_118,MFCC_119,MFCC_120,MFCC_121,MFCC_122,MFCC_123,MFCC_124,MFCC_125,MFCC_126,MFCC_127
0,1.206752,0.161699,0.223483,0.111338,0.728755,1.411419,1.541712,-1.156927,-1.768883,0.294013,...,2.244230,2.892967,2.158813,-1.804063,-1.603646,-0.318842,3.154804,-0.022835,-1.090258,-2.480426
1,-3.044144,0.919848,-0.330244,0.067806,0.155189,0.373561,-0.182510,-1.239840,0.687802,-1.241711,...,-0.477666,-1.652310,-1.360360,-0.066186,1.240867,0.920321,0.101280,-0.600844,0.415870,0.013912
2,0.672800,-1.114655,1.497597,-1.373886,-0.531722,-1.005042,-0.425740,-4.219006,5.945963,-0.994139,...,0.987076,-0.483030,-0.648311,-1.187675,0.300975,1.444555,0.448793,-0.867031,-1.600708,-1.925837
3,0.563289,0.443466,0.838457,0.185989,0.446582,0.960872,2.140261,2.079478,1.491859,0.601933,...,-0.264568,1.844388,-0.296166,0.284225,-0.717624,-1.053334,-0.315267,-0.005973,-1.728596,0.221867
4,1.007568,-2.369496,-4.098495,-1.901266,-2.775209,-2.779459,-3.348817,-2.859062,-2.709460,-3.587404,...,-0.412924,0.197805,-0.365878,1.547914,-0.587778,-1.665059,-0.778595,-0.124566,-0.180980,-0.045812


In [375]:
def get_mfcc_delta_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y,
                                    sr=sr,
                                    n_mfcc=128,
                                    dct_type=2)
        
        mfcc = librosa.feature.delta(mfcc,
                                     order=1)

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:

            e = np.mean(e)

            y_feature.append(e)

        features.append(y_feature)

    columns = ['MFCC__delta_'+str(i) for i in range(len(features[0]))]
    
    mfcc_df = pd.DataFrame(features,
                           columns=columns)

    print(mfcc_df.shape)

    return mfcc_df

In [376]:
mfcc_delta_train = get_mfcc_delta_feature(df_train)
mfcc_delta_test = get_mfcc_delta_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 128)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 128)


In [377]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
mfcc_delta_train, mfcc_delta_test= scaled_df(mfcc_delta_train,
                                 df_train,
                                 mfcc_delta_test,
                                 df_test,
                                 scaler)

mfcc_delta_train.columns = ['MFCC_delta_'+str(i) for i in range(len(mfcc_delta_train.columns))]
mfcc_delta_test.columns = ['MFCC_delta_'+str(i) for i in range(len(mfcc_delta_test.columns))]

In [378]:
mfcc_delta_train.head()

,MFCC_delta_0,MFCC_delta_1,MFCC_delta_2,MFCC_delta_3,MFCC_delta_4,MFCC_delta_5,MFCC_delta_6,MFCC_delta_7,MFCC_delta_8,MFCC_delta_9,...,MFCC_delta_118,MFCC_delta_119,MFCC_delta_120,MFCC_delta_121,MFCC_delta_122,MFCC_delta_123,MFCC_delta_124,MFCC_delta_125,MFCC_delta_126,MFCC_delta_127
0,-0.076830,-1.496073,-0.440771,-0.184562,-0.047025,1.458084,0.865210,1.175551,0.991063,-0.693079,...,-2.880890,0.947400,0.430416,2.505769,0.905387,1.539257,2.122332,0.482585,-0.399781,-1.056024
1,-0.482420,-2.083301,0.295557,-0.705273,0.957594,-1.117608,-0.127952,0.670008,-0.389617,-1.810475,...,0.432090,-0.681884,-1.253939,-0.598928,-1.122006,1.143832,2.093967,0.488574,-0.512215,-0.349107
2,-0.751712,-0.666003,0.112888,0.867451,1.208890,0.792049,0.458680,0.961787,0.527333,-0.541283,...,-1.861980,-0.546202,0.887007,4.383711,3.846259,-0.746569,-1.222766,0.287204,-3.537661,-0.102120
3,-1.051355,-0.669949,0.358324,2.407805,0.617582,0.857858,1.062858,1.258847,-1.839074,0.884781,...,0.644993,-0.790503,-0.274649,-0.954199,-1.331004,0.700375,1.025289,0.994299,-0.334136,0.120222
4,-2.951004,-1.813607,1.479423,1.398908,-0.241960,-1.066584,0.315026,-2.118484,0.329192,-0.938180,...,1.023238,-0.028922,-1.056836,2.073604,2.047538,-2.408021,-2.504472,0.228138,1.234958,1.041292


### Chroma

* 모든 음표는 12개의 pitch(음높이)와 octave로 구성되어 있다.

* Chroma는 각 audio frame의 octave는 무시하고, 12개의 pitch class에 대한 분포를 나타낸 것이다.

* Pitch class는 C, C#, D, D#, E, F, F#, G, G#, A, A#, B로 구성되어 있다.

#### Chroma stft

* Compute a chromagram from a waveform or power spectrogram.

In [526]:
def get_chroma_stft_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_stft(y=y, sr=sr,
                                             n_chroma=12)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_stft_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [527]:
chroma_stft_train = get_chroma_stft_feature(df_train)
chroma_stft_test = get_chroma_stft_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 12)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 12)


In [528]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
chroma_stft_train, chroma_stft_test= scaled_df(chroma_stft_train,
                                     df_train,
                                     chroma_stft_test,
                                     df_test,
                                     scaler)

chroma_stft_train.columns = ['Chroma_stft_'+str(i) for i in range(len(chroma_stft_train.columns))]
chroma_stft_test.columns = ['Chroma_stft_'+str(i) for i in range(len(chroma_stft_test.columns))]

In [529]:
chroma_stft_train.head()

,Chroma_stft_0,Chroma_stft_1,Chroma_stft_2,Chroma_stft_3,Chroma_stft_4,Chroma_stft_5,Chroma_stft_6,Chroma_stft_7,Chroma_stft_8,Chroma_stft_9,Chroma_stft_10,Chroma_stft_11
0,-1.070049,1.012718,3.325309,3.093195,-1.021851,-2.577379,2.523919,2.374777,0.352113,-1.919175,0.731797,-0.096159
1,-1.142371,-0.549684,0.481691,0.741929,-0.183350,0.027769,-0.564759,-0.913597,-0.587752,-0.290928,0.576997,-0.297999
2,-1.888211,-2.316258,-1.216508,-0.754540,7.915114,11.200861,1.859251,-1.701285,-1.856211,-1.886382,-2.083567,-2.128482
3,-1.152342,-1.299468,-1.681248,-1.467963,1.299843,0.420721,-0.698317,-1.247342,-0.863656,0.009655,-1.903455,-1.355041
4,0.457912,-0.945649,-1.236708,-1.411892,-0.795418,0.692870,1.140367,1.178353,2.821418,1.487543,1.078149,1.005651


In [600]:
def get_chroma_stft_delta_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_stft(y=y, sr=sr,
                                             n_chroma=12)
        
        chroma = librosa.feature.delta(chroma,
                                       order=1)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_stft_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [602]:
chroma_stft_delta_train = get_chroma_stft_delta_feature(df_train)
chroma_stft_delta_test = get_chroma_stft_delta_feature(df_test)

  0%|          | 0/1279 [00:00<?, ?it/s]

(1279, 12)


  0%|          | 0/1514 [00:00<?, ?it/s]

(1514, 12)


In [603]:
scaler = RobustScaler(quantile_range=(35.0, 65.0))
chroma_stft_delta_train, chroma_stft_delta_test = scaled_df(chroma_stft_delta_train,
                                     df_train,
                                     chroma_stft_delta_test,
                                     df_test,
                                     scaler)

chroma_stft_delta_train.columns = ['Chroma_stft_'+str(i) for i in range(len(chroma_stft_delta_train.columns))]
chroma_stft_delta_test.columns = ['Chroma_stft_'+str(i) for i in range(len(chroma_stft_delta_test.columns))]

In [604]:
chroma_stft_delta_train.head()

,Chroma_stft_0,Chroma_stft_1,Chroma_stft_2,Chroma_stft_3,Chroma_stft_4,Chroma_stft_5,Chroma_stft_6,Chroma_stft_7,Chroma_stft_8,Chroma_stft_9,Chroma_stft_10,Chroma_stft_11
0,1.272769,-0.459903,1.925715,0.716492,0.234022,-0.281618,-0.693816,-1.939308,-1.853648,-0.233747,0.503726,0.823124
1,1.151142,2.176059,1.242870,0.525835,-0.606254,-3.010945,-2.374622,-0.310494,-0.992796,-0.362564,0.857919,0.858160
2,-0.433666,-1.774703,-1.030115,-0.359814,-2.429800,-0.114101,5.419895,2.494764,1.082360,0.394856,0.779679,-0.108635
3,0.285595,0.109167,-0.343420,-0.703956,-0.016538,0.606383,0.001230,-1.114259,-1.448370,-1.126101,-0.033244,0.526520
4,1.787625,1.596735,0.729618,-0.504742,0.792314,0.719976,-0.448218,0.314747,-0.233679,-1.037499,-0.580268,1.222587


#### Chroma cqt

* Constant-Q chromagram

In [ ]:
def get_chroma_cpt_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_cqt(y=y, sr=sr,
                                             n_chroma=12)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_cqt_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [ ]:
chroma_cqt_train = get_chroma_cpt_feature(df_train)
chroma_cqt_test = get_chroma_cpt_feature(df_test)

In [ ]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
chroma_cqt_train, chroma_cqt_test= scaled_df(chroma_cqt_train,
                                     df_train,
                                     chroma_cqt_test,
                                     df_test,
                                     scaler)

chroma_cqt_train.columns = ['Chroma_cqt_'+str(i) for i in range(len(chroma_cqt_train.columns))]
chroma_cqt_test.columns = ['Chroma_cqt_'+str(i) for i in range(len(chroma_cqt_test.columns))]

In [ ]:
chroma_cqt_train.head()

#### Chroma cens

* Computes the chroma variant “Chroma Energy Normalized” (CENS)

* To compute CENS features, following steps are taken after obtaining chroma vectors using chroma_cqt: 1.

  - L-1 normalization of each chroma vector

  - Quantization of amplitude based on “log-like” amplitude thresholds

  - (optional) Smoothing with sliding window. Default window length = 41 frames

  - (not implemented) Downsampling

* CENS features are robust to dynamics, timbre and articulation, thus these are commonly used in audio matching and retrieval applications.

In [ ]:
def get_chroma_cens_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)
        
        # librosa패키지를 사용하여 Chroma 추출
        chroma = librosa.feature.chroma_cens(y=y, sr=sr,
                                             n_chroma=12)

        y_feature = []
        # 추출된 Chroma들의 평균을 Feature로 사용
        for e in chroma:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Chroma_cens_'+str(i) for i in range(len(features[0]))]

    chroma_df = pd.DataFrame(features,
                             columns=columns)
    
    print(chroma_df.shape)

    return chroma_df

In [ ]:
chroma_cens_train = get_chroma_cens_feature(df_train)
chroma_cens_test = get_chroma_cens_feature(df_test)

In [ ]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
chroma_cens_train, chroma_cens_test= scaled_df(chroma_cens_train,
                                     df_train,
                                     chroma_cens_test,
                                     df_test,
                                     scaler)

chroma_cens_train.columns = ['Chroma_cens_'+str(i) for i in range(len(chroma_cens_train.columns))]
chroma_cens_test.columns = ['Chroma_cens_'+str(i) for i in range(len(chroma_cens_test.columns))]

### Tonnetz

* Computes the tonal centroid features (tonnetz)

* This representation uses the method of to project chroma features onto a 6-dimensional basis representing the perfect fifth, minor third, and major third each as two-dimensional coordinates.

* 1739년 오일러에 의해 처음 기술된 것으로 음악의 tonality와 tonal space의 관계를 graphical한 방식으로 표현한 것

In [ ]:
def get_tonnetz_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)

        # librosa패키지를 사용하여 chroma 추출
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12)
        #chroma = librosa.feature.chroma_cqt(y=y, sr=sr, n_chroma=12)
        #chroma = librosa.feature.chroma_cens(y=y, sr=sr, n_chroma=12)
        
        # librosa패키지를 사용하여 tonnetz 추출
        tonnetz = librosa.feature.tonnetz(y=y, sr=sr,
                                          n_chroma=12,
                                          chroma=chroma)

        y_feature = []
        # 추출된 tonnetz들의 평균을 Feature로 사용
        for e in tonnetz:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Tonnetz_'+str(i) for i in range(len(features[0]))]
    
    tonnetz_df = pd.DataFrame(features,
                           columns=columns)

    print(tonnetz_df.shape)

    return tonnetz_df

In [ ]:
tonnetz_train = get_tonnetz_feature(df_train)
tonnetz_test = get_tonnetz_feature(df_test)

In [ ]:
scaler = RobustScaler()
tonnetz_train, tonnetz_test= scaled_df(tonnetz_train,
                                       df_train,
                                       tonnetz_test,
                                       df_test,
                                       scaler)

tonnetz_train.columns = ['Tonnetz_'+str(i) for i in range(len(tonnetz_train.columns))]
tonnetz_test.columns = ['Tonnetz_'+str(i) for i in range(len(tonnetz_test.columns))]

In [ ]:
tonnetz_train.head()

### Rhythm

* Compute the tempogram: local autocorrelation of the onset strength envelope.

In [ ]:
def get_rhythm_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=16000)

        # librosa패키지를 사용하여 onset 추출
        onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
        #onset_envelope = librosa.onset.onset_detect(y=y, sr=sr)
        #onset_envelope = librosa.onset.onset_strength_multi(y=y, sr=sr)
        
        # librosa패키지를 사용하여 Rhythm 추출
        rhythm = librosa.feature.tempogram(y=y, sr=sr,
                                           onset_envelope=onset_envelope)

        y_feature = []
        # 추출된 tonnetz들의 평균을 Feature로 사용
        for e in rhythm:

            y_feature.append(np.mean(e))

        features.append(y_feature)

    columns = ['Rhythm'+str(i) for i in range(len(features[0]))]
    
    rhythm_df = pd.DataFrame(features,
                           columns=columns)

    print(rhythm_df.shape)

    return rhythm_df

In [ ]:
rhythm_train = get_rhythm_feature(df_train)
rhythm_test = get_rhythm_feature(df_test)

In [ ]:
scaler = RobustScaler(quantile_range=(40.0, 60.0))
rhythm_train, rhythm_test= scaled_df(rhythm_train,
                                       df_train,
                                       rhythm_test,
                                       df_test,
                                       scaler)

rhythm_train.columns = ['Rhythm_'+str(i) for i in range(len(rhythm_train.columns))]
rhythm_test.columns = ['Rhythm_'+str(i) for i in range(len(rhythm_test.columns))]

In [ ]:
rhythm_train.head()

## Dimension Reduction

### Truncated SVD

### PCA

In [ ]:
pca = PCA()
pca.fit(melspectrogram_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.99999) + 1
print(N_COMPONETS)

127


In [ ]:
pca = PCA(n_components=N_COMPONETS)

pca_train_mel = pca.fit_transform(melspectrogram_train)
pca_test_mel = pca.transform(melspectrogram_test)

pca_train_mel = pd.DataFrame(pca_train_mel)
pca_test_mel = pd.DataFrame(pca_test_mel)

In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_mel = pca.fit_transform(melspectrogram_train)
pca_test_mel = pca.transform(melspectrogram_test)

pca_train_mel = pd.DataFrame(pca_train_mel)
pca_test_mel = pd.DataFrame(pca_test_mel)

#### Rhythm

In [ ]:
pca = PCA()
pca.fit(rhythm_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

119


In [ ]:
pca = PCA(n_components=N_COMPONETS)

pca_train_rhythm = pca.fit_transform(rhythm_train)
pca_test_rhythm = pca.transform(rhythm_test)

pca_train_rhythm = pd.DataFrame(pca_train_rhythm)
pca_test_rhythm = pd.DataFrame(pca_test_rhythm)

### Sparse PCA

#### Zero Crossing Rate

In [379]:
pca = PCA()
pca.fit(zero_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [380]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_zero = pca.fit_transform(zero_train)
pca_test_zero = pca.transform(zero_test)

pca_train_zero = pd.DataFrame(pca_train_zero)
pca_test_zero = pd.DataFrame(pca_test_zero)

In [381]:
pca = PCA()
pca.fit(zero_delta_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [382]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_zero_delta = pca.fit_transform(zero_delta_train)
pca_test_zero_delta = pca.transform(zero_delta_test)

pca_train_zero_delta = pd.DataFrame(pca_train_zero_delta)
pca_test_zero_delta = pd.DataFrame(pca_test_zero_delta)

#### RMS

In [383]:
pca = PCA()
pca.fit(rms_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [384]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_rms = pca.fit_transform(rms_train)
pca_test_rms = pca.transform(rms_test)

pca_train_rms = pd.DataFrame(pca_train_rms)
pca_test_rms = pd.DataFrame(pca_test_rms)

In [385]:
pca = PCA()
pca.fit(rms_delta_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [386]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_rms_delta = pca.fit_transform(rms_delta_train)
pca_test_rms_delta = pca.transform(rms_delta_test)

pca_train_rms_delta = pd.DataFrame(pca_train_rms_delta)
pca_test_rms_delta = pd.DataFrame(pca_test_rms_delta)

#### MFCC

In [387]:
pca = PCA()
pca.fit(mfcc_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

124


In [388]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_mfcc = pca.fit_transform(mfcc_train)
pca_test_mfcc = pca.transform(mfcc_test)

pca_train_mfcc = pd.DataFrame(pca_train_mfcc)
pca_test_mfcc = pd.DataFrame(pca_test_mfcc)

In [447]:
pca = PCA()
pca.fit(mfcc_delta_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.85) + 1
print(N_COMPONETS)

78


In [515]:
# 3
pca = SparsePCA(n_components=3,
                alpha=0.001)

pca_train_mfcc_delta = pca.fit_transform(mfcc_delta_train)
pca_test_mfcc_delta = pca.transform(mfcc_delta_test)

pca_train_mfcc_delta = pd.DataFrame(pca_train_mfcc_delta)
pca_test_mfcc_delta = pd.DataFrame(pca_test_mfcc_delta)

#### Spectral


##### Spectral Centroid

In [ ]:
pca = PCA()
pca.fit(spectral_centroid_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_centroid = pca.fit_transform(spectral_centroid_train)
pca_test_sepctral_centroid = pca.transform(spectral_centroid_test)

pca_train_sepctral_centroid = pd.DataFrame(pca_train_sepctral_centroid)
pca_test_sepctral_centroid = pd.DataFrame(pca_test_sepctral_centroid)

##### Spectral Bandwidth

In [ ]:
pca = PCA()
pca.fit(spectral_bandwidth_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_bandwidth = pca.fit_transform(spectral_bandwidth_train)
pca_test_sepctral_bandwidth = pca.transform(spectral_bandwidth_test)

pca_train_sepctral_bandwidth = pd.DataFrame(pca_train_sepctral_bandwidth)
pca_test_sepctral_bandwidth = pd.DataFrame(pca_test_sepctral_bandwidth)

##### Spectral Contrast

In [ ]:
pca = PCA()
pca.fit(spectral_contrast_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

7


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_contrast = pca.fit_transform(spectral_contrast_train)
pca_test_sepctral_contrast = pca.transform(spectral_contrast_test)

pca_train_sepctral_contrast = pd.DataFrame(pca_train_sepctral_contrast)
pca_test_sepctral_contrast = pd.DataFrame(pca_test_sepctral_contrast)

##### Spectral Flatness


In [ ]:
pca = PCA()
pca.fit(spectral_flatness_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_flatness = pca.fit_transform(spectral_flatness_train)
pca_test_sepctral_flatness = pca.transform(spectral_flatness_test)

pca_train_sepctral_flatness = pd.DataFrame(pca_train_sepctral_flatness)
pca_test_sepctral_flatness = pd.DataFrame(pca_test_sepctral_flatness)

##### Spectral Rolloff

In [ ]:
pca = PCA()
pca.fit(spectral_rolloff_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

1


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_sepctral_rolloff = pca.fit_transform(spectral_rolloff_train)
pca_test_sepctral_rolloff = pca.transform(spectral_rolloff_test)

pca_train_sepctral_rolloff = pd.DataFrame(pca_train_sepctral_rolloff)
pca_test_sepctral_rolloff = pd.DataFrame(pca_test_sepctral_rolloff)

#### Chroma

##### Chroma stft

In [566]:
pca = PCA()
pca.fit(chroma_stft_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.90) + 1
print(N_COMPONETS)

8


In [683]:
# 3
pca = SparsePCA(n_components=1,
                alpha=0.001)

pca_train_chroma_stft = pca.fit_transform(chroma_stft_train)
pca_test_chroma_stft = pca.transform(chroma_stft_test)

pca_train_chroma_stft = pd.DataFrame(pca_train_chroma_stft)
pca_test_chroma_stft = pd.DataFrame(pca_test_chroma_stft)

In [684]:
pca = PCA()
pca.fit(chroma_stft_delta_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.90) + 1
print(N_COMPONETS)

8


In [685]:
pca = SparsePCA(n_components=1,
                alpha=0.001)

pca_train_chroma_stft_delta = pca.fit_transform(chroma_stft_delta_train)
pca_test_chroma_stft_delta = pca.transform(chroma_stft_delta_test)

pca_train_chroma_stft_delta = pd.DataFrame(pca_train_chroma_stft_delta)
pca_test_chroma_stft_delta = pd.DataFrame(pca_test_chroma_stft_delta)

##### Chroma cqt

In [ ]:
pca = PCA()
pca.fit(chroma_cqt_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

12


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_chroma_cqt = pca.fit_transform(chroma_cqt_train)
pca_test_chroma_cqt = pca.transform(chroma_cqt_test)

pca_train_chroma_cqt = pd.DataFrame(pca_train_chroma_cqt)
pca_test_chroma_cqt = pd.DataFrame(pca_test_chroma_cqt)

##### Chroma cens

In [ ]:
pca = PCA()
pca.fit(chroma_cens_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

12


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_chroma_cens = pca.fit_transform(chroma_cens_train)
pca_test_chroma_cens = pca.transform(chroma_cens_test)

pca_train_chroma_cens = pd.DataFrame(pca_train_chroma_cens)
pca_test_chroma_cens = pd.DataFrame(pca_test_chroma_cens)

### Tonnetz

In [ ]:
pca = PCA()
pca.fit(tonnetz_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

6


In [ ]:
pca = SparsePCA(n_components=N_COMPONETS,
                alpha=0.001)

pca_train_tonnetz = pca.fit_transform(tonnetz_train)
pca_test_tonnetz = pca.transform(tonnetz_test)

pca_train_tonnetz = pd.DataFrame(pca_train_tonnetz)
pca_test_tonnetz = pd.DataFrame(pca_test_tonnetz)

### Kernel PCA

#### Poly Feature

In [ ]:
pca = PCA()
pca.fit(poly_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
N_COMPONETS = np.argmax(cumsum>=0.999) + 1
print(N_COMPONETS)

2


In [ ]:
pca = KernelPCA(n_components=N_COMPONETS)

pca_train_poly = pca.fit_transform(poly_train)
pca_test_poly = pca.transform(poly_test)

pca_train_poly = pd.DataFrame(pca_train_poly)
pca_test_poly = pd.DataFrame(pca_test_poly)

# Concat Preprocessed Data Set

In [686]:
preprocessed_train_mean = pd.concat([
                                     #zero_train,
                                     #rms_train,
                                     #pca_train_mel,
                                     pca_train_zero,
                                     pca_train_rms,
                                     pca_train_mfcc,
                                     pca_train_zero_delta,
                                     pca_train_rms_delta,
                                     #pca_train_mfcc_delta,
                                     #pca_train_sepctral_centroid,
                                     #pca_train_sepctral_bandwidth,
                                     #pca_train_sepctral_contrast,
                                     #pca_train_sepctral_flatness,
                                     #pca_train_sepctral_rolloff,
                                     pca_train_chroma_stft,
                                     pca_train_chroma_stft_delta,
                                     #pca_train_chroma_cqt,
                                     #pca_train_chroma_cens,
                                     #pca_train_tonnetz,
                                     #pca_train_rhythm,
                                     #pca_train_poly
                                    ], axis=1)

preprocessed_test_mean = pd.concat([
                                     #zero_test,
                                     #rms_test,
                                     #pca_test_mel,
                                     pca_test_zero,
                                     pca_test_rms,
                                     pca_test_mfcc,
                                     pca_test_zero_delta,
                                     pca_test_rms_delta,
                                     #pca_test_mfcc_delta,
                                     #pca_test_sepctral_centroid,
                                     #pca_test_sepctral_bandwidth,
                                     #pca_test_sepctral_contrast,
                                     #pca_test_sepctral_flatness,
                                     #pca_test_sepctral_rolloff,
                                     pca_test_chroma_stft,
                                     pca_test_chroma_stft_delta,
                                     #pca_test_chroma_cqt,
                                     #pca_test_chroma_cens,
                                     #pca_test_tonnetz,
                                     #pca_test_rhythm,
                                     #pca_test_poly
                                    ], axis=1)

In [687]:
preprocessed_train_mean.columns = [i for i in range(len(preprocessed_train_mean.columns))]
preprocessed_test_mean.columns = [i for i in range(len(preprocessed_test_mean.columns))]

In [688]:
print(preprocessed_train_mean.shape)
preprocessed_train_mean.head()

(1279, 130)


,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,129
0,-0.303446,0.983766,3.422420,0.726486,-2.462682,4.443269,-5.738622,-1.975347,3.681233,12.694907,...,-0.549037,0.153809,-0.179582,0.076368,0.010871,-0.241233,-0.210859,-1.261785,0.738058,-0.488538
1,-0.899772,-0.012764,2.502096,2.519140,-1.577035,-0.769271,3.478615,1.338044,-0.147000,-0.281568,...,0.102917,0.163011,0.536513,-0.826751,0.172047,-0.250567,-0.382049,-0.275590,-0.234001,-0.440172
2,0.438706,-1.466346,-4.175946,2.231185,1.994829,1.515495,-3.933909,3.124208,6.541959,-1.330611,...,0.058560,-0.574130,0.047098,-0.028901,0.352297,0.002886,0.671386,-0.522044,-8.460591,2.525541
3,-0.901006,1.523705,0.084066,1.280448,0.559491,-0.635935,1.115205,-1.207013,4.503405,2.572394,...,-0.171057,0.029320,-0.077531,-0.599213,-0.102897,0.148886,1.243426,-0.659879,-3.277138,-1.091706
4,1.995991,-1.468288,-1.570391,0.251557,0.460797,2.018924,3.932621,-2.159932,-7.148360,-2.894867,...,-0.189837,0.629114,-0.088787,0.012352,0.752945,-0.023018,-1.509588,-1.704008,1.523623,0.784361


# Modeling

## LOF

### 모델 생성

In [689]:
model = LocalOutlierFactor(n_neighbors=1,
                           algorithm='auto',
                           contamination='auto',
                           novelty=True)

### 모델 학습

In [690]:
model.fit(preprocessed_train_mean)

LocalOutlierFactor(n_neighbors=1, novelty=True)

### 추론

In [691]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량) 이므로 (0:정상, 1:불량)로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [692]:
test_pred = model.predict(preprocessed_test_mean) 
test_pred = get_pred_label(test_pred)

# Submission

In [693]:
submit = pd.read_csv('./sample_submission.csv')

In [694]:
submit['LABEL'] = test_pred
submit.head()

,SAMPLE_ID,LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,1


In [695]:
submit['LABEL'].value_counts()

1    817
0    697
Name: LABEL, dtype: int64

In [355]:
submit.to_csv('./submit110.csv', index=False)